In [ ]:
%load_ext autoreload
%autoreload 2
from awesome.run.awesome_config import AwesomeConfig
from awesome.run.awesome_runner import AwesomeRunner
from awesome.util.reflection import class_name
from awesome.analytics.result_model import ResultModel
from awesome.util.path_tools import get_project_root_path, get_package_root_path
import os
import torch
import re
from awesome.util.format import latex_postprocessor

os.chdir(get_project_root_path()) # Beeing in the root directory of the project is important for the relative paths to work consistently

In [ ]:
from awesome.analytics.result_comparison import ResultComparison

paths = ['./runs/unireps/joint','./runs/unireps/individual' ]
models = []

for path in paths:
    for folder in os.listdir(path):
        model = ResultModel.from_path(os.path.join(path, folder))
        models.append(model)

import re
p = r"#?(?P<cfg_num>\d+)?_?(?P<net>\w+)_benchmark(?P<feat>(\+\w+)*)\_(?P<date>\d{2}_\d{2}_\d{2})\_(?P<time>\d{2}_\d{2}_\d{2})"
pattern = re.compile(p)

for model in models:
    match = pattern.match(model.name)
    model_name = None
    feat = []
    if match:
        model_name = match.group('net')
        features = match.group('feat')
        if features is not None and features != "":
            feat = features.strip("+").split("+")
            if not any(["seed" in x for x in feat]):
                feat.append("seed42")
            feat = sorted(feat)
    else:
        print('No match for', model.name)
    model_name = model_name.replace("NET", "Net")
    model.display_name = model_name + " " + " ".join(feat)
    model.config.result_directory = "final_mask"
    model.save_config()


# Resort the models by name to get a meaningful table order

_order = []

models = sorted(models, key=lambda m: _order.index(m.name) if m.name in _order else 0)

comparison = ResultComparison(models)
comparison.assign_numbers(force=True)

os.environ['PLOT_OUTPUT_DIR'] = comparison.output_folder

save_args = dict(transparent=False, save=True, dpi=300, ext=["png", "pdf"])



In [ ]:
# TODO Recompute the mIoU for the final masks
from awesome.agent.torch_agent_checkpoint import TorchAgentCheckpoint
from awesome.agent.util.tracker import Tracker
from awesome.measures.miou import MIOU
import awesome.run.functions as f
from awesome.util.temporary_property import TemporaryProperty

model = models[0]
recompute_models = models #[model] # models
image_idx = 0
fig = None

miou_bg = MIOU(noneclass=2., average="binary", name="MeanBackgroundBinaryMIOU") 
miou_fg = MIOU(noneclass=2., average="binary", invert=True, name="MeanForegroundBinaryMIOU")

miou_bg_prior = MIOU(noneclass=2., average="binary", name="MeanPriorBackgroundBinaryMIOU") 
miou_fg_prior = MIOU(noneclass=2., average="binary", invert=True, name="MeanPriorForegroundBinaryMIOU")

fg_miou_res = None
bg_miou_res = None
fg_prior_miou_res = None
bg_prior_miou_res = None
for model in recompute_models:
    model: ResultModel
    with TemporaryProperty(model, getitem_mask_mode="both"):
        runner = model.get_runner(-1)
        tracker: Tracker = model.get_tracker(-1)
        dataloader = runner.dataloader
        with TemporaryProperty(dataloader, dimension="3d"):
            indices = list(range(len(dataloader)))
            
            fg_image_iou = torch.zeros(len(indices))
            bg_image_iou = torch.zeros(len(indices))

            fg_prior_image_iou = torch.zeros(len(indices))
            bg_prior_image_iou = torch.zeros(len(indices))

            for image_idx in indices:
                image, ground_truth, _input, fg, bg, prior_state = f.prepare_input_eval(dataloader, None, image_idx)
                predicted_mask, prior_mask = model[image_idx]
                bg_ground_truth = ground_truth
                fg_ground_truth = 1-ground_truth

                bg_predicted_mask = 1 - predicted_mask
                bg_prior_mask = 1 - prior_mask

                fg_miou_res = miou_fg(bg_predicted_mask, bg_ground_truth)
                fg_image_iou[image_idx] = fg_miou_res

                bg_miou_res = miou_bg(bg_predicted_mask, bg_ground_truth)
                bg_image_iou[image_idx] = bg_miou_res

                fg_prior_miou_res = miou_fg(bg_prior_mask, bg_ground_truth)
                fg_prior_image_iou[image_idx] = fg_prior_miou_res

                bg_prior_miou_res = miou_bg(bg_prior_mask, bg_ground_truth)
                bg_prior_image_iou[image_idx] = bg_prior_miou_res

                
                #fig = f.plot_mask(image, bg_predicted_mask, background_value=1., title=f"BG MIOU: {bg_miou_res:.3f}, FG {fg_miou_res:.3f}")
                #fig_2 = f.plot_mask(image, bg_prior_mask, background_value=1., title=f"BG MIOU: {bg_prior_miou_res:.3f}, FG {fg_prior_miou_res:.3f}")

            mean_fg_image_iou = fg_image_iou.mean()
            mean_fg_prior_image_iou = fg_prior_image_iou.mean()
            mean_bg_image_iou = bg_image_iou.mean()
            mean_bg_prior_image_iou = bg_prior_image_iou.mean()

            print(model.config.name)
            print(f"FG Miou Seg:{mean_fg_image_iou} Prior:{mean_fg_prior_image_iou}")
            print(f"BG Miou Seg:{mean_bg_image_iou} Prior:{mean_bg_prior_image_iou}")

            old_bg_seg_miou = tracker.metrics['eval/epoch/MeanBinaryMIOU'].values.iloc[-1].value
            old_bg_prio_miou = tracker.metrics['eval/epoch/MeanPriorBinaryMIOU'].values.iloc[-1].value

            assert mean_bg_image_iou == old_bg_seg_miou, f"Segment does not align for: {model.config.name}"
            assert mean_bg_prior_image_iou == old_bg_prio_miou, f"Prior does not align for: {model.config.name}"

            tracker.create_epoch_metric(miou_fg, handle_exists="ignore")
            tracker.create_epoch_metric(miou_bg, handle_exists="ignore")
            tracker.create_epoch_metric(miou_fg_prior, handle_exists="ignore")
            tracker.create_epoch_metric(miou_bg_prior, handle_exists="ignore")

            tracker.epoch_metric(Tracker.get_metric_name(miou_fg), mean_fg_image_iou, False, step=tracker.global_epochs)
            tracker.epoch_metric(Tracker.get_metric_name(miou_bg), mean_bg_image_iou, False, step=tracker.global_epochs)
            tracker.epoch_metric(Tracker.get_metric_name(miou_fg_prior), mean_fg_prior_image_iou, False, step=tracker.global_epochs)
            tracker.epoch_metric(Tracker.get_metric_name(miou_bg_prior), mean_bg_prior_image_iou, False, step=tracker.global_epochs)

            dfs = [tracker.metrics['eval/epoch/MeanForegroundBinaryMIOU'].values,
            tracker.metrics['eval/epoch/MeanPriorForegroundBinaryMIOU'].values,
            tracker.metrics['eval/epoch/MeanBackgroundBinaryMIOU'].values,
            tracker.metrics['eval/epoch/MeanPriorBackgroundBinaryMIOU'].values]
            
        checkpoint: TorchAgentCheckpoint = model.get_checkpoint(-1)
        checkpoint.tracker = tracker
        eps = tracker.global_epochs
        new_path = os.path.join(model.output_directory, model.CHECKPOINT_FORMAT.format(eps))
        print(new_path)
        checkpoint.save(new_path)
